**Sinergia Dev** playbook

# Openai Responses API

- python install requirements

In [ ]:
!pip install openai

In [ ]:
import getpass
import os

# set env for openai key:

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

Enter API key for OpenAI: ··········


In [ ]:
from openai import OpenAI
client = OpenAI()

# Generate text from a simple prompt

## ChatGPT LLM single call
response = client.responses.create(
    model="gpt-4o-mini",
    input="Escribe un cuento de buenas noches en una sola oración sobre un unicornio."
)

## print response output text
print("response.output_text:\n")
print(response.output_text)

## debug full response object
print("response object:\n")
pprint.pprint(response.model_dump())

In [ ]:
# Generate text with instructions

## ChatGPT LLM single call
response = client.responses.create(
    model="gpt-4o",
    instructions="Your goals is to translate to english every user input",
    input="Escribe un cuento de buenas noches en una sola oración sobre un unicornio.",
)

## print response output text
print("response.output_text:\n")
print(response.output_text)

## debug full response object
print("response object:\n")
pprint.pprint(response.model_dump())

In [ ]:
# Generate text with messages using different roles

## ChatGPT LLM with message history style (it will preserve instructions on history)
response = client.responses.create(
    model="gpt-4o-mini",
    input=[
        {
            "role": "developer",
            "content": "Your goals is to translate to spanish every user input"
        },
        {
            "role": "user",
            "content": "Are semicolons optional in JavaScript?"
        }
    ]
)

## print response output text
print("response.output_text:\n")
print(response.output_text)

## debug full response object
print("response object:\n")
pprint.pprint(response.model_dump())

# Structured Outputs

In [ ]:
import json

# structured output models support by:

# gpt-4.5-preview-2025-02-27 and later
# o3-mini-2025-1-31 and later
# o1-2024-12-17 and later
# gpt-4o-mini-2024-07-18 and later
# gpt-4o-2024-08-06 and later

response = client.responses.create(
    model="gpt-4o-2024-08-06",
    input=[
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": "Matias and Jony are going to SinergIA-Dev on 25 Friday, march at 15hs."}
    ],
    text={
        "format": {
            "type": "json_schema",
            "name": "calendar_event",
            "schema": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string"
                    },
                    "date": {
                        "type": "string"
                    },
                    "participants": {
                        "type": "array",
                        "items": {
                            "type": "string"
                        }
                    },
                },
                "required": ["name", "date", "participants"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
)

event = json.loads(response.output_text)

print(event)

{'name': 'SinergIA-Dev', 'date': '2023-03-25T15:00:00', 'participants': ['Matias', 'Jony']}


In [ ]:
import json
import pprint

# Structured Outputs for chain-of-thought math tutoring

response = client.responses.create(
    model="gpt-4o-2024-08-06",
    input=[
        {"role": "system", "content": "You are a helpful math tutor. Guide the user through the solution step by step."},
        {"role": "user", "content": "how can I solve 8x + 7 = -23"}
    ],
    text={
        "format": {
            "type": "json_schema",
            "name": "math_reasoning",
            "schema": {
                "type": "object",
                "properties": {
                    "steps": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "explanation": { "type": "string" },
                                "output": { "type": "string" }
                            },
                            "required": ["explanation", "output"],
                            "additionalProperties": False
                        }
                    },
                    "final_answer": { "type": "string" }
                },
                "required": ["steps", "final_answer"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
)

math_reasoning = json.loads(response.output_text)
pprint.pprint(math_reasoning)


{'final_answer': 'x = -\\frac{15}{4}',
 'steps': [{'explanation': "Start by isolating the term with the variable 'x'. "
                           'We need to move the constant term on the left side '
                           'to the right side. To do this, subtract 7 from '
                           'both sides of the equation.',
            'output': '8x + 7 - 7 = -23 - 7'},
           {'explanation': 'This simplifies to:', 'output': '8x = -30'},
           {'explanation': "Next, solve for 'x' by dividing both sides of the "
                           'equation by 8.',
            'output': '\\( x = \\frac{-30}{8} \\)'},
           {'explanation': 'Simplify the fraction by dividing the numerator '
                           'and the denominator by their greatest common '
                           'divisor, which is 2.',
            'output': '\\( x = \\frac{-15}{4} \\)'},
           {'explanation': 'Thus, the solution to the equation is:',
            'output': 'x = -\\frac{15}{4

In [ ]:
# https://python.langchain.com/docs/concepts/prompt_templates/
# install langchain

!pip install langchain
!pip install -qU "langchain[openai]"

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage

# define model to use
model = init_chat_model("gpt-4o-mini", model_provider="openai")

# message preparation:

## using history messages:
messages = [
    SystemMessage("Translate the following from English into Spanish"),
    HumanMessage("hi! Im a developer trying to learn AI"),
]

# LLM call

## invoke chat completion
response = model.invoke(messages)

# print response text
print("response.content:\n")
print(response.content + "\n")
print("response object:\n")
pprint.pprint(response.model_dump())


In [ ]:
# example for a dynamic template completion

## using template with variables
system_template = "Translate the following from English into {language}" # variable on template interpolated with {var_name}

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

prompt = prompt_template.invoke({"language": "Italian", "text": "hi! Please take the recommended actions to update your configuration"})



In [ ]:
template_prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")

chain = template_prompt | model
response1 = chain.invoke({"topic": "programming"})
print(response1.content)

Why do programmers prefer dark mode?  

Because light attracts bugs!


In [ ]:
import pprint

pprint.pprint(response1.model_dump())


# Composed LLM (chaining)

- pedir un boceto de algo al llm
- pedir una critica del boceto
- pedir algo al llm incluyendo el boceto y la critica.



---



# Agents: Human in the loop

In [ ]:
# https://github.com/agno-agi/agno

In [ ]:
!pip install httpx rich agno

In [ ]:
"""🤝 Human-in-the-Loop: Adding User Confirmation to Tool Calls

This example shows how to implement human-in-the-loop functionality in your Agno tools.
It shows how to:
- Add pre-hooks to tools for user confirmation
- Handle user input during tool execution
- Gracefully cancel operations based on user choice

Some practical applications:
- Confirming sensitive operations before execution
- Reviewing API calls before they're made
- Validating data transformations
- Approving automated actions in critical systems

Run `pip install openai httpx rich agno` to install dependencies.
"""

import json
from textwrap import dedent
from typing import Iterator

import httpx
from agno.agent import Agent
from agno.exceptions import StopAgentRun
from agno.tools import FunctionCall, tool
from rich.console import Console
from rich.pretty import pprint
from rich.prompt import Prompt

# This is the console instance used by the print_response method
# We can use this to stop and restart the live display and ask for user confirmation
console = Console()


def pre_hook(fc: FunctionCall):
    # Get the live display instance from the console
    live = console._live

    # Stop the live display temporarily so we can ask for user confirmation
    live.stop()  # type: ignore

    # Ask for confirmation
    console.print(f"\nAbout to run [bold blue]{fc.function.name}[/]")
    message = (
        Prompt.ask("Do you want to continue?", choices=["y", "n"], default="y")
        .strip()
        .lower()
    )

    # Restart the live display
    live.start()  # type: ignore

    # If the user does not want to continue, raise a StopExecution exception
    if message != "y":
        raise StopAgentRun(
            "Tool call cancelled by user",
            agent_message="Stopping execution as permission was not granted.",
        )


@tool(pre_hook=pre_hook)
def get_top_hackernews_stories(num_stories: int) -> Iterator[str]:
    """Fetch top stories from Hacker News after user confirmation.

    Args:
        num_stories (int): Number of stories to retrieve

    Returns:
        str: JSON string containing story details
    """
    # Fetch top story IDs
    response = httpx.get("https://hacker-news.firebaseio.com/v0/topstories.json")
    story_ids = response.json()

    # Yield story details
    for story_id in story_ids[:num_stories]:
        story_response = httpx.get(
            f"https://hacker-news.firebaseio.com/v0/item/{story_id}.json"
        )
        story = story_response.json()
        if "text" in story:
            story.pop("text", None)
        yield json.dumps(story)


# Initialize the agent with a tech-savvy personality and clear instructions
agent = Agent(
    description="A Tech News Assistant that fetches and summarizes Hacker News stories",
    instructions=dedent("""\
        You are an enthusiastic Tech Reporter

        Your responsibilities:
        - Present Hacker News stories in an engaging and informative way
        - Provide clear summaries of the information you gather

        Style guide:
        - Use emoji to make your responses more engaging
        - Keep your summaries concise but informative
        - End with a friendly tech-themed sign-off\
    """),
    tools=[get_top_hackernews_stories],
    show_tool_calls=True,
    markdown=True,
)

# Example questions to try:
# - "What are the top 3 HN stories right now?"
# - "Show me the most recent story from Hacker News"
# - "Get the top 5 stories (you can try accepting and declining the confirmation)"
agent.print_response(
    "Show me a list of title from top 5 most recent story from Hacker News", stream=True, console=console
)

# View all messages
# pprint(agent.run_response.messages)

Output()

About to run get_top_hackernews_stories

Do you want to continue? [y/n] (y):



---



# Memory w/mem0

In [ ]:
# https://github.com/mem0ai/mem0

In [ ]:
!pip install mem0ai

In [ ]:
from openai import OpenAI
from mem0 import Memory

openai_client = OpenAI()
memory = Memory()

def chat_with_memories(message: str, user_id: str = "default_user") -> str:
    # Retrieve relevant memories
    relevant_memories = memory.search(query=message, user_id=user_id, limit=3)
    memories_str = "\n".join(f"- {entry['memory']}" for entry in relevant_memories["results"])

    # Generate Assistant response
    system_prompt = f"You are a helpful AI. Answer the question based on query and memories.\nUser Memories:\n{memories_str}"
    messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": message}]
    # Completion API
    response = openai_client.chat.completions.create(model="gpt-4o-mini", messages=messages)
    assistant_response = response.choices[0].message.content

    # Create new memories from the conversation
    messages.append({"role": "assistant", "content": assistant_response})
    memory.add(messages, user_id=user_id)

    return assistant_response

def main():
    print("Chat with AI (type 'exit' to quit)")
    while True:
        user_input = input("You: ").strip()
        if user_input.lower() == 'exit':
            print("Goodbye!")
            break
        print(f"AI: {chat_with_memories(user_input)}")

if __name__ == "__main__":
    main()

Chat with AI (type 'exit' to quit)
You: hola me llamo Matias y tengo 39 años
AI: Hola Matías, ¡es un placer conocerte! ¿En qué puedo ayudarte hoy?
You: uh perdon creo que te dije mal mi edad
AI: No hay problema, Matías. ¿Cuál es tu edad correcta?
You: que edad te dije?
AI: Me dijiste que tienes 39 años.
You: tengo 40
AI: ¡Feliz cumpleaños, Matías! Ahora tienes 40 años. ¿Hay algo especial que te gustaría hacer para celebrar este nuevo año?
You: exit
Goodbye!
